In [1]:
!pip install -q langchain_community
!pip install -q chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which

In [2]:
import pandas as pd
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
import os

# Step 1: Data Loading

In [3]:
excel_path="/kaggle/input/contentcertificate/2025.xlsx"

df = pd.read_excel(excel_path)

df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,القطاع Segment,رقم القطاع الرئيسي\nSegment No,اسم القطاع الرئيسي(عربي)\nSegment Title (Arabic),اسم القطاع الرئيسي(إنجليزي)\nSegment Title (English),الرمز في منصة اعتماد\nEtimad Code,اسم المنتج (عربي)\nCommodity Title (Arabic),اسم المنتج (انجليزي)\nCommodity Title (English),وصف المنتج (عربي)\nCommodity Definition (Arabic),وصف المنتج (انجليزي)\nCommodity Definition (English),السقف السعري\nPrice Ceiling,تاريخ التطبيق\nEffective Date,الحد الأدنى لخط الأساس لمصنع المنتج\nManufacturer Local Content Minimum Baseline
0,البناء والتشييد,3.0,الخرسانة,CONCRETE,2001,مواد تسليح الخرسانة,Concrete Reinforcement Materials,يتم تعريف هذا المنتج على أنه: مواد تسليح لتعزي...,This product is defined as : Reinforcement to ...,0.3,2019-12-09,يشترط وجود شهادة المحتوى المحلي للمصنع
1,NaN,NaN,NaN,NaN,2002,خرسانة مصبوبة في الموقع,Cast-in-Place Concrete,يتم تعريف هذا المنتج على أنه: تقنية بناء المب...,This product is defined as : a technology of ...,0.15,2019-12-09,يشترط وجود شهادة المحتوى المحلي للمصنع
2,NaN,NaN,NaN,NaN,2003,الخرسانة مسبقة الصب,Precast Concrete,يتم تعريف هذا المنتج على أنه: منتجات الخرسانة ...,This product is defined as : Construction prod...,0.15,2019-12-09,يشترط وجود شهادة المحتوى المحلي للمصنع
3,NaN,NaN,NaN,NaN,2004,الخرسانة الحجمية العادية,Mass Concrete,يتم تعريف هذا المنتج على أنه: أي حجم من الخرسا...,This product is defined as: Large volume place...,0.15,2019-12-09,يشترط وجود شهادة المحتوى المحلي للمصنع
4,NaN,NaN,NaN,NaN,2005,الخرسانة المسلحة بالألياف الزجاجية,Glass Fiber Reinforced Concrete (GRC),يتم تعريف هذا المنتج على أنه: ألواح خرسانية مع...,This product defined as : Panels are for decor...,0.25,2019-12-09,يشترط وجود شهادة المحتوى المحلي للمصنع


# Step 2: Data Preprocessing

In [4]:
df.columns

Index([' القطاع Segment ', 'رقم القطاع الرئيسي\nSegment No',
       'اسم القطاع الرئيسي(عربي)\nSegment Title (Arabic)',
       'اسم القطاع الرئيسي(إنجليزي)\nSegment Title (English)',
       'الرمز في منصة اعتماد\nEtimad Code',
       'اسم المنتج (عربي)\nCommodity Title (Arabic)',
       'اسم المنتج (انجليزي)\nCommodity Title (English)',
       'وصف المنتج (عربي)\nCommodity Definition (Arabic)',
       'وصف المنتج (انجليزي)\nCommodity Definition (English)',
       'السقف السعري\nPrice Ceiling', 'تاريخ التطبيق\nEffective Date',
       'الحد الأدنى لخط الأساس لمصنع المنتج\nManufacturer Local Content Minimum Baseline'],
      dtype='object')

In [5]:
# Columns to fill down
columns_to_fill = list(df.columns[:4])

# Apply forward fill (ffill) to propagate the values downwards
df[columns_to_fill] = df[columns_to_fill].ffill()

# Display the head to verify
df.head(10)

,القطاع Segment,رقم القطاع الرئيسي\nSegment No,اسم القطاع الرئيسي(عربي)\nSegment Title (Arabic),اسم القطاع الرئيسي(إنجليزي)\nSegment Title (English),الرمز في منصة اعتماد\nEtimad Code,اسم المنتج (عربي)\nCommodity Title (Arabic),اسم المنتج (انجليزي)\nCommodity Title (English),وصف المنتج (عربي)\nCommodity Definition (Arabic),وصف المنتج (انجليزي)\nCommodity Definition (English),السقف السعري\nPrice Ceiling,تاريخ التطبيق\nEffective Date,الحد الأدنى لخط الأساس لمصنع المنتج\nManufacturer Local Content Minimum Baseline
0,البناء والتشييد,3.0,الخرسانة,CONCRETE,2001,مواد تسليح الخرسانة,Concrete Reinforcement Materials,يتم تعريف هذا المنتج على أنه: مواد تسليح لتعزي...,This product is defined as : Reinforcement to ...,0.3,2019-12-09,يشترط وجود شهادة المحتوى المحلي للمصنع
1,البناء والتشييد,3.0,الخرسانة,CONCRETE,2002,خرسانة مصبوبة في الموقع,Cast-in-Place Concrete,يتم تعريف هذا المنتج على أنه: تقنية بناء المب...,This product is defined as : a technology of ...,0.15,2019-12-09,يشترط وجود شهادة المحتوى المحلي للمصنع
2,البناء والتشييد,3.0,الخرسانة,CONCRETE,2003,الخرسانة مسبقة الصب,Precast Concrete,يتم تعريف هذا المنتج على أنه: منتجات الخرسانة ...,This product is defined as : Construction prod...,0.15,2019-12-09,يشترط وجود شهادة المحتوى المحلي للمصنع
3,البناء والتشييد,3.0,الخرسانة,CONCRETE,2004,الخرسانة الحجمية العادية,Mass Concrete,يتم تعريف هذا المنتج على أنه: أي حجم من الخرسا...,This product is defined as: Large volume place...,0.15,2019-12-09,يشترط وجود شهادة المحتوى المحلي للمصنع
4,البناء والتشييد,3.0,الخرسانة,CONCRETE,2005,الخرسانة المسلحة بالألياف الزجاجية,Glass Fiber Reinforced Concrete (GRC),يتم تعريف هذا المنتج على أنه: ألواح خرسانية مع...,This product defined as : Panels are for decor...,0.25,2019-12-09,يشترط وجود شهادة المحتوى المحلي للمصنع
5,البناء والتشييد,4.0,مواد أعمال البناء,MASONRY,2006,أعمال البناء الحراري: مواد البناء الحرارية,Refractory Masonry: Refractory Masonry Materials,يتم تعريف هذا المنتج على أنه: مواد البناء الحر...,This product is defined as : Refractory masonr...,0.15,2019-12-09,يشترط وجود شهادة المحتوى المحلي للمصنع
6,البناء والتشييد,4.0,مواد أعمال البناء,MASONRY,2007,وحدات البناء والطوب,Masonry and Blocks Unit,يتم تعريف هذا المنتج على أنه: وحدات البناء ، ا...,This product is defined as: Masonry unit const...,0.15,2019-12-09,يشترط وجود شهادة المحتوى المحلي للمصنع
7,البناء والتشييد,4.0,مواد أعمال البناء,MASONRY,2008,أعمال البناء المقاوم للتآكل: مواد أعمال البناء...,Corrosion-Resistant Masonry: Corrosion-Resista...,يتم تعريف هذا المنتج على أنه: مواد أعمال البنا...,This product is defined as : Masonry construct...,0.15,2019-12-09,يشترط وجود شهادة المحتوى المحلي للمصنع
8,البناء والتشييد,4.0,مواد أعمال البناء,MASONRY,2009,أعمال البناء المصنعة: مواد البناء المصنعة,Manufactured Masonry: Manufactured Masonry Mat...,يتم تعريف هذا المنتج على أنه: وحدات البناء الس...,This product is defined as : Hydraulically com...,0.15,2019-12-09,يشترط وجود شهادة المحتوى المحلي للمصنع
9,البناء والتشييد,5.0,المعادن,METALS,2010,الإطارات المعدنية المسحوبة على البارد: مواد تأ...,Cold-Formed Metal Framing: Cold-Formed Metal F...,يتم تعريف هذا المنتج على أنه: عنصر معدني هيكل...,This product is defined as : Structured metal ...,0.3,2019-12-09,يشترط وجود شهادة المحتوى المحلي للمصنع


In [6]:
df.columns[[5,6,-1]]

Index(['اسم المنتج (عربي)\nCommodity Title (Arabic)',
       'اسم المنتج (انجليزي)\nCommodity Title (English)',
       'الحد الأدنى لخط الأساس لمصنع المنتج\nManufacturer Local Content Minimum Baseline'],
      dtype='object')

In [16]:
def load_and_prepare_data(df: pd.DataFrame) -> list[dict]:

    documents_ar = []
    documents_en = []

    col_metadata = df.columns[-1] # Metadata column (Manufacturer Local Content Minimum Baseline)
    col_ar, col_en = df.columns[5], df.columns[6]
    
    for index, row in df.iterrows():

        value_metadata = row.get(col_metadata)
        metadata = {col_metadata: None if pd.isna(value_metadata) else value_metadata}
        
        value_ar = row.get(col_ar, '')
        value_en = row.get(col_en, '')
        page_content_ar = str(value_ar).strip()
        page_content_en = str(value_en).strip()
        
        documents_ar.append({"page_content": page_content_ar, "metadata": metadata})
        documents_en.append({"page_content": page_content_en, "metadata": metadata})

    return documents_ar, documents_en

# Step 3: Prepare documents for the vector store

In [25]:
prepared_docs_ar, prepared_docs_en = load_and_prepare_data(df)
print(f"Arabic has: #{len(prepared_docs_ar)}")
print(prepared_docs_ar[:5], end = "\n\n\n")

print(f"English has: #{len(prepared_docs_en)}")
print(prepared_docs_en[:5], end = "\n\n\n")

Arabic has: #1444
[{'page_content': 'مواد تسليح الخرسانة', 'metadata': {'الحد الأدنى لخط الأساس لمصنع المنتج\nManufacturer Local Content Minimum Baseline': 'يشترط وجود شهادة المحتوى المحلي للمصنع'}}, {'page_content': 'خرسانة مصبوبة في الموقع', 'metadata': {'الحد الأدنى لخط الأساس لمصنع المنتج\nManufacturer Local Content Minimum Baseline': 'يشترط وجود  شهادة المحتوى المحلي للمصنع'}}, {'page_content': 'الخرسانة مسبقة الصب', 'metadata': {'الحد الأدنى لخط الأساس لمصنع المنتج\nManufacturer Local Content Minimum Baseline': 'يشترط وجود  شهادة المحتوى المحلي للمصنع'}}, {'page_content': 'الخرسانة الحجمية العادية', 'metadata': {'الحد الأدنى لخط الأساس لمصنع المنتج\nManufacturer Local Content Minimum Baseline': 'يشترط وجود  شهادة المحتوى المحلي للمصنع'}}, {'page_content': 'الخرسانة المسلحة بالألياف الزجاجية', 'metadata': {'الحد الأدنى لخط الأساس لمصنع المنتج\nManufacturer Local Content Minimum Baseline': 'يشترط وجود  شهادة المحتوى المحلي للمصنع'}}]


English has: #1444
[{'page_content': 'Concrete

# Step 4: Initialize the vector store

In [27]:
def initialize_vector_store(documents: list[dict], DB_DIR):

    # Initialize the embedding model
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

    # Check if the vector store directory exists and contains data
    # Note: ChromaDB's persistence check can be more robust, but this is a common starting point
    if os.path.exists(DB_DIR) and os.path.isdir(DB_DIR) and len(os.listdir(DB_DIR)) > 0:
        print("Loading existing vector store from disk.")
        vectordb = Chroma(persist_directory=DB_DIR, embedding_function=embeddings)
    else:
        print("Vector store not found or empty, creating and persisting new one.")
        
        texts = [doc['page_content'] for doc in documents]
        metadatas = [doc['metadata'] for doc in documents]
        
        vectordb = Chroma.from_texts(
            texts=texts,
            embedding=embeddings,
            metadatas=metadatas,
            persist_directory=DB_DIR
        )
        vectordb.persist() # Ensure the database is written to disk
        print("Vector store created and persisted.")
        
    return vectordb

In [28]:
# Define the directory for persisting the ChromaDB for English
DB_DIR_en = "/kaggle/working/chroma_db_data_english"
# Define the directory for persisting the ChromaDB for Arabic
DB_DIR_ar = "/kaggle/working/chroma_db_data_Arabic"

vector_db_ar = initialize_vector_store(prepared_docs_ar, DB_DIR_ar)
vector_db_en = initialize_vector_store(prepared_docs_en, DB_DIR_en)

/tmp/ipykernel_36/3291284671.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
2025-07-23 08:48:26.548954: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753260506.739486      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753260506.792880      36 cuda_blas.cc:1418] Unable to register cuBLAS fac

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector store not found or empty, creating and persisting new one.


/tmp/ipykernel_36/3291284671.py:23: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist() # Ensure the database is written to disk


Vector store created and persisted.
Vector store not found or empty, creating and persisting new one.
Vector store created and persisted.


In [30]:
import shutil
import os

# Define the path to the ChromaDB directory
DB_DIR = "/kaggle/working/chroma_db_data_english"
# Define the name for the zip file (without .zip extension)
ZIP_NAME = "chroma_db_archive_en"
# Define the full path for the output zip file
OUTPUT_ZIP_PATH = f"/kaggle/working/{ZIP_NAME}"

if os.path.exists(DB_DIR):
    print(f"Zipping the ChromaDB folder: {DB_DIR}...")
    # Create a zip archive of the directory
    shutil.make_archive(OUTPUT_ZIP_PATH, 'zip', DB_DIR)
    print(f"ChromaDB folder successfully zipped to {OUTPUT_ZIP_PATH}.zip")
    print("You can now download this file from the Kaggle 'Output' section or by clicking on the file in the file browser.")
else:
    print(f"Error: ChromaDB directory '{DB_DIR}' not found. Please ensure it was created successfully.")

Zipping the ChromaDB folder: /kaggle/working/chroma_db_data_english...
ChromaDB folder successfully zipped to /kaggle/working/chroma_db_archive_en.zip
You can now download this file from the Kaggle 'Output' section or by clicking on the file in the file browser.


In [ ]:
import shutil
import os

# Define the path to the ChromaDB directory
DB_DIR = "/kaggle/working/chroma_db_data_Arabic"
# Define the name for the zip file (without .zip extension)
ZIP_NAME = "chroma_db_archive_ar"
# Define the full path for the output zip file
OUTPUT_ZIP_PATH = f"/kaggle/working/{ZIP_NAME}"

if os.path.exists(DB_DIR):
    print(f"Zipping the ChromaDB folder: {DB_DIR}...")
    # Create a zip archive of the directory
    shutil.make_archive(OUTPUT_ZIP_PATH, 'zip', DB_DIR)
    print(f"ChromaDB folder successfully zipped to {OUTPUT_ZIP_PATH}.zip")
    print("You can now download this file from the Kaggle 'Output' section or by clicking on the file in the file browser.")
else:
    print(f"Error: ChromaDB directory '{DB_DIR}' not found. Please ensure it was created successfully.")